# Desafio da Semana Três - Acelera Dev DS

## Importando bibliotecas e dataset

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

In [50]:
data = pd.read_csv('./desafio1.csv')

## Vendo informações básicas sobre o dataset

In [3]:
data.head()

,RowNumber,id,sobrenome,pontuacao_credito,estado_residencia,genero,idade,nivel_estabilidade,saldo_conta,numero_produtos,possui_cartao_de_credito,membro_ativo
0,1,e7f44fcbd380d4cef7e6c232cc7e37895c3fd197,6d6e0aa1b9b413e442e2fb68df14b4fc3f91de50,619,SC,F,42,2,0.00,1,1,1
1,2,28dcb083ad90512da16b9430085c2cddb8ca5e12,48e1ad846796fa314f1b4a6702b83343eb5482c5,608,RS,F,41,1,83807.86,1,0,1
2,3,774bc378f787438c9c7594e536787d07a097a54b,f2b29d5d934de615812b697132e767dea0f1e9e2,502,SC,F,42,8,159660.80,3,1,0
3,4,043a71326f7096de155e7f0c559dc62b5e4b7239,85d5c9da7cddd8109ad32a6c348fe2bb10bf99f2,699,SC,F,39,1,0.00,2,0,0
4,5,37b3bb8ca243955fb3605ec7e1c2a4607cdb3b22,ddc89c837a6933639de75f28171057060bd322df,850,RS,F,43,2,125510.82,1,1,1


In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 12 columns):
RowNumber                   7000 non-null int64
id                          7000 non-null object
sobrenome                   7000 non-null object
pontuacao_credito           7000 non-null int64
estado_residencia           7000 non-null object
genero                      7000 non-null object
idade                       7000 non-null int64
nivel_estabilidade          7000 non-null int64
saldo_conta                 7000 non-null float64
numero_produtos             7000 non-null int64
possui_cartao_de_credito    7000 non-null int64
membro_ativo                7000 non-null int64
dtypes: float64(1), int64(7), object(4)
memory usage: 656.4+ KB


## Agrupando as métricas por estado

In [24]:
estados = data['estado_residencia'].value_counts().index

In [52]:
#PEgar Moda, mediana, média e desvio padrão pra cada estado
grouped_data = data.groupby('estado_residencia')['pontuacao_credito']\
                     .agg([('mediana', 'median'),('media','mean'),('desvio_padrao','std'),\
                            ('moda',lambda estado:estado.value_counts().index[0])])\
                     .reset_index()
grouped_data

,estado_residencia,mediana,media,desvio_padrao,moda
0,PR,650,648.961294,98.607186,850
1,RS,650,651.105143,95.136598,850
2,SC,653,649.537653,97.233493,850


## Vendo arquivo para submissão do desafio

In [53]:
submission = pd.read_json('./submission.json')
submission.head()

,SC,RS,PR
moda,850.000000,850.000000,850.000000
mediana,653.000000,650.000000,650.000000
media,649.537653,651.105143,648.961294
desvio_padrao,97.233493,95.136598,98.607186


## Ajustando dados ao formato certo para submissão

In [41]:
melted_data = grouped_data.melt(id_vars="estado_residencia", 
        var_name="metricas", 
        value_name="pontuacao_credito")
melted_data

,estado_residencia,metricas,pontuacao_credito
0,PR,mediana,650.000000
1,RS,mediana,650.000000
2,SC,mediana,653.000000
3,PR,media,648.961294
4,RS,media,651.105143
5,SC,media,649.537653
6,PR,desvio_padrao,98.607186
7,RS,desvio_padrao,95.136598
8,SC,desvio_padrao,97.233493
9,PR,moda,850.000000


In [54]:
pivot_data = melted_data.pivot(index='metricas',columns='estado_residencia',\
                              values='pontuacao_credito')
pivot_data

estado_residencia,PR,RS,SC
metricas,,,
desvio_padrao,98.607186,95.136598,97.233493
media,648.961294,651.105143,649.537653
mediana,650.000000,650.000000,653.000000
moda,850.000000,850.000000,850.000000


## Ajuste final e preenchimento do df de submissão

In [46]:
final_data = pivot_data.reindex(index=pivot_data.index[::-1])
final_data

estado_residencia,PR,RS,SC
metricas,,,
moda,850.000000,850.000000,850.000000
mediana,650.000000,650.000000,653.000000
media,648.961294,651.105143,649.537653
desvio_padrao,98.607186,95.136598,97.233493


In [55]:
for estado in estados:
    submission[estado] = final_data[estado]

## gravando os dados no arquivo de submissão

In [49]:
submission.to_json('./submission.json')